In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pymongo
from pymongo import MongoClient

In [2]:
# Connection to mongo
client = MongoClient('mongodb+srv://<username>:<password>@cluster0.l3pqt.mongodb.net/MSA?retryWrites=true&w=majority')
# Select database
db = client['MSA']
# see list of collections
client.MSA.list_collection_names()

['arima_LaborParticipation_pred_score',
 'CPI_RegionUrban_raw',
 'arima_pop_2019_2024',
 'BusinessApplications_clean',
 'arima_GDP_2019_2024',
 'employment_prediction',
 'Employment_clean',
 'CPI_RegionUrban_clean',
 'population_predicted_2024',
 'employment_clean',
 'LaborForce_clean',
 'GDP_raw',
 'unemployment_ROC_2024',
 'employment_raw',
 'OES_raw',
 'Population_clean',
 'unemployment_predicted_2024',
 'arima_pop_pred_score',
 'msa_codes',
 'arima_unem_pred_score',
 'population_raw',
 'arima_emp_pred_score',
 'LaborForce_raw',
 'arima_cpi_pred_score',
 'UnemploymentRate_clean',
 'Employment_raw',
 'arima_2024_ROC_score_total',
 'regions_divisions',
 'arima_business_pred_score',
 'LaborParticipation_clean',
 'unemployment_raw',
 'AnnualMeanWage_clean',
 'GDP_clean',
 'UnemploymentRate_raw',
 'Predicted_2024_ROC_rank_total',
 'arima_GDP_pred_score',
 'Predicted_2024_ROC_rank_total2',
 'GDP_predicted_2024']

In [3]:
# Select the collection with needed data
emp = db.Employment_clean
df = pd.DataFrame(list(emp.find()))
print(df.dtypes)
df

_id     object
CBSA     int64
2010     int64
2011     int64
2012     int64
2013     int64
2014     int64
2015     int64
2016     int64
2017     int64
2018     int64
2019     int64
dtype: object


,_id,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,603dba7e680af8db371b1e2e,10180,71455,72162,72210,72877,72614,71810,71786,73630,74942,77346
1,603dba7e680af8db371b1e2f,10420,333843,333177,333872,333060,341435,340759,340621,342135,344267,343479
2,603dba7e680af8db371b1e30,10500,62624,63468,62213,61378,61231,62158,64476,64247,63869,64581
3,603dba7e680af8db371b1e31,10540,49886,49832,48453,48629,50348,52528,54219,55353,56139,56468
4,603dba7e680af8db371b1e32,10580,417173,415614,422290,422015,419228,426623,423587,422072,428744,433120
...,...,...,...,...,...,...,...,...,...,...,...,...
379,603dba7e680af8db371b1fa9,49420,100992,100872,100803,99456,103473,104395,108536,111456,114141,118684
380,603dba7e680af8db371b1faa,49620,212564,214874,216426,216079,219070,222944,223961,224295,226766,228723
381,603dba7e680af8db371b1fab,49660,238069,239089,237392,235580,237477,234932,230271,227637,227149,223658
382,603dba7e680af8db371b1fac,49700,60165,61217,61259,61659,62462,64438,64836,67064,67306,72021


In [4]:
df.drop(columns='_id', inplace=True)
df

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,71455,72162,72210,72877,72614,71810,71786,73630,74942,77346
1,10420,333843,333177,333872,333060,341435,340759,340621,342135,344267,343479
2,10500,62624,63468,62213,61378,61231,62158,64476,64247,63869,64581
3,10540,49886,49832,48453,48629,50348,52528,54219,55353,56139,56468
4,10580,417173,415614,422290,422015,419228,426623,423587,422072,428744,433120
...,...,...,...,...,...,...,...,...,...,...,...
379,49420,100992,100872,100803,99456,103473,104395,108536,111456,114141,118684
380,49620,212564,214874,216426,216079,219070,222944,223961,224295,226766,228723
381,49660,238069,239089,237392,235580,237477,234932,230271,227637,227149,223658
382,49700,60165,61217,61259,61659,62462,64438,64836,67064,67306,72021


In [5]:
# for loop to predict 2024 values
prediction2020 = []
prediction2021 = []
prediction2022 = []
prediction2023 = []
prediction2024 = []
for i in range(0,384):
    y = df.iloc[i, 1:].values
    series = pd.Series(y, dtype='int')
    model = ARIMA(series, order=(2, 1, 1))
    model_fit = model.fit()
    pred = model_fit.forecast(5)
    forecast = pred.values.tolist()
    prediction2020.append(forecast[0])
    prediction2021.append(forecast[1])
    prediction2022.append(forecast[2])
    prediction2023.append(forecast[3])
    prediction2024.append(forecast[4])
df['2020'] = prediction2020
df['2021'] = prediction2021
df['2022'] = prediction2022
df['2023'] = prediction2023
df['2024'] = prediction2024

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: Convergen

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

In [6]:
df

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,10180,71455,72162,72210,72877,72614,71810,71786,73630,74942,77346,77974.094187,78594.369256,79214.458726,79834.395592,80454.180031
1,10420,333843,333177,333872,333060,341435,340759,340621,342135,344267,343479,343598.742903,343480.746745,343598.652857,343480.832611,343598.567063
2,10500,62624,63468,62213,61378,61231,62158,64476,64247,63869,64581,64618.931153,64585.186264,64571.915908,64569.835128,64570.023389
3,10540,49886,49832,48453,48629,50348,52528,54219,55353,56139,56468,56354.718633,56468.280200,56355.640308,56467.372667,56356.540567
4,10580,417173,415614,422290,422015,419228,426623,423587,422072,428744,433120,433101.620301,433102.201995,433101.692995,433102.201697,433101.692998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,49420,100992,100872,100803,99456,103473,104395,108536,111456,114141,118684,120719.003398,122856.390971,124989.517631,127122.740284,129255.880935
380,49620,212564,214874,216426,216079,219070,222944,223961,224295,226766,228723,228753.193405,228751.492388,228753.601321,228751.498837,228753.600814
381,49660,238069,239089,237392,235580,237477,234932,230271,227637,227149,223658,221933.544493,220274.490252,218651.925026,217064.601899,215511.748064
382,49700,60165,61217,61259,61659,62462,64438,64836,67064,67306,72021,73266.570218,74567.640237,75867.812703,77167.989950,78468.157540


In [7]:
df['2020'] = df['2020'].astype('int')
df['2021'] = df['2021'].astype('int')
df['2022'] = df['2022'].astype('int')
df['2023'] = df['2023'].astype('int')
df['2024'] = df['2024'].astype('int')
df.head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,10180,71455,72162,72210,72877,72614,71810,71786,73630,74942,77346,77974,78594,79214,79834,80454
1,10420,333843,333177,333872,333060,341435,340759,340621,342135,344267,343479,343598,343480,343598,343480,343598
2,10500,62624,63468,62213,61378,61231,62158,64476,64247,63869,64581,64618,64585,64571,64569,64570
3,10540,49886,49832,48453,48629,50348,52528,54219,55353,56139,56468,56354,56468,56355,56467,56356
4,10580,417173,415614,422290,422015,419228,426623,423587,422072,428744,433120,433101,433102,433101,433102,433101


In [8]:
df.sort_values(by='2024').head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
110,21420,26736,27816,28807,27808,28869,28844,27358,26412,26279,25898,25897,25834,25866,25839,25859
57,16180,23057,22740,22756,22699,22705,22865,23399,24366,24519,25433,25642,25940,26227,26515,26804
363,47460,28669,28472,28133,27061,27491,27608,28164,28421,29687,30609,30786,30679,30777,30690,30768
88,19180,33667,33860,32891,31724,32705,32371,31762,31633,31157,31426,31531,31536,31522,31515,31514
201,30300,27930,27961,28251,28125,29081,29320,30366,30454,30283,30999,31329,31702,32071,32440,32809


In [9]:
# create new collection in mongo
arima_emp_2019_2024 = db.arima_emp_2019_2024

In [10]:
df_dict = df.to_dict(orient='records')

In [11]:
arima_emp_2019_2024.insert_many(df_dict)